<a href="https://colab.research.google.com/github/lisaangelicamanente-ux/LLM-course-2025/blob/main/week-2/in-context-learning/in_context_learning_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In-Context Learning


In-context learning is a generalisation of few-shot learning where the LLM is provided a context as part of the prompt and asked to respond by utilising the information in the context.

* Example: *"Summarize this research article into one paragraph highlighting its strengths and weaknesses: [insert article text]”*
* Example: *"Extract all the quotes from this text and organize them in alphabetical order: [insert text]”*

A very popular technique that you will learn in week 5 called Retrieval-Augmented Generation (RAG) is a form of in-context learning, where:
* a search engine is used to retrieve some relevant information
* that information is then provided to the LLM as context


In this example we download some recent research papers from arXiv papers, extract the text from the PDF files and ask Gemini to summarize the articles as well as provide the main strengths and weaknesses of the papers. Finally we print the summaries to a local html file and as markdown.

In [25]:
!pip install requests bs4 google-generativeai pypdf

In [26]:
import os
import requests
from bs4 import BeautifulSoup
#import google.generativeai as genai
from urllib.request import urlopen, urlretrieve
from IPython.display import Markdown, display
from pypdf import PdfReader
from datetime import date
from tqdm import tqdm
from google.colab import userdata

In [6]:
#API_KEY = os.environ.get("nuova")
#API_KEY = userdata.get('nuova')
#genai.configure(api_key=API_KEY)

We select those papers that have been featured in Hugging Face papers.

In [27]:
BASE_URL = "https://huggingface.co/papers"
page = requests.get(BASE_URL)
soup = BeautifulSoup(page.content, "html.parser")
h3s = soup.find_all("h3")

papers = []

for h3 in h3s:
    a = h3.find("a")
    title = a.text
    link = a["href"].replace('/papers', '')

    papers.append({"title": title, "url": f"https://arxiv.org/pdf{link}"})
    papers = papers[:3]

Code to extract text from PDFs.

In [28]:
def extract_paper(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text


def extract_pdf(url):
    pdf = urlretrieve(url, "pdf_file.pdf")
    reader = PdfReader("pdf_file.pdf")
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

def extract_pdf_limited(url, max_chars=12000):
    text = extract_pdf(url)
    return text[:max_chars]

text = extract_pdf_limited(paper["url"])

def printmd(string):
    display(Markdown(string))

In [29]:
#LLM = "gemini-2.5-flash"
#model = genai.GenerativeModel(LLM)

from transformers import pipeline

LLM = "google/flan-t5-base"


generator = pipeline(
    "text2text-generation",
    model=LLM,
    device=0
)

Device set to use cuda:0


We use Gemini to summarize the papers.

In [ ]:
#for paper in tqdm(papers):
    #try:
     #   paper["summary"] = model.generate_content("Summarize this research article into one paragraph without formatting highlighting its strengths and weaknesses. " + extract_pdf(paper["url"])).text
    #except:
     #   print("Generation failed")
      #  paper["summary"] = "Paper not available"

Using an open-source Hugging Face model to summarize the papers.

In [30]:
papers = papers[:3]

for paper in tqdm(papers):
    print("Starting generation...")

    prompt = (
    "Task: Read the research paper below.\n\n"
    "List exactly TWO strengths and TWO weaknesses.\n"
    "Use the following format exactly:\n\n"
    "Strengths:\n"
    "- item 1\n"
    "- item 2\n\n"
    "Weaknesses:\n"
    "- item 1\n"
    "- item 2\n\n"
    "Paper:\n"
    + extract_pdf_limited(paper["url"], max_chars=2500)
)


    result = generator(
        prompt,
        max_new_tokens=180,
        do_sample=False
    )

    paper["summary"] = result[0]["generated_text"]
    print("Done.")

  0%|          | 0/3 [00:00<?, ?it/s]

Starting generation...


Token indices sequence length is longer than the specified maximum sequence length for this model (719 > 512). Running this sequence through the model will result in indexing errors
 33%|███▎      | 1/3 [00:03<00:07,  3.90s/it]

Done.
Starting generation...


 67%|██████▋   | 2/3 [00:07<00:03,  3.70s/it]

Done.
Starting generation...


100%|██████████| 3/3 [00:13<00:00,  4.39s/it]

Done.


In [31]:
import re

for paper in papers:
    text = paper["summary"]

    strengths = ""
    weaknesses = ""

    if "Strengths" in text:
        strengths = text.split("Strengths", 1)[1]

    if "Weaknesses" in strengths:
        strengths, weaknesses = strengths.split("Weaknesses", 1)

    strengths = strengths.replace(":", "").strip()
    weaknesses = weaknesses.replace(":", "").strip()

    if strengths == "":
        strengths = "- Not clearly identified"

    if weaknesses == "":
        weaknesses = "- Not clearly identified"

    paper["table"] = f"""
| Strengths | Weaknesses |
|----------|------------|
| {strengths.replace('\n', '<br>')} | {weaknesses.replace('\n', '<br>')} |
"""

In [32]:
for paper in papers:
    printmd(f"**[{paper['title']}]({paper['url']})**\n\n{paper['table']}")


**[T-pro 2.0: An Efficient Russian Hybrid-Reasoning Model and Playground](https://arxiv.org/pdf/2512.10430)**


| Strengths | Weaknesses |
|----------|------------|
| - item 1 - item 2 | - item 1 - item 2 |


**[Are We Ready for RL in Text-to-3D Generation? A Progressive Investigation](https://arxiv.org/pdf/2512.10949)**


| Strengths | Weaknesses |
|----------|------------|
| - item 1 - item 2 | - item 1 - item 2 |


**[Long-horizon Reasoning Agent for Olympiad-Level Mathematical Problem Solving](https://arxiv.org/pdf/2512.10739)**


| Strengths | Weaknesses |
|----------|------------|
| - item 1 - item 2 | - item 1 - item 2 Paper 2025-12-12 Long-horizon Reasoning Agent for Olympiad-Level Mathematical Problem Solving Songyang Gao1* Yuzhe Gu1,2* Zijian Wu1,3* Lingkai Kong1,2* Wenwei Zhang1* Zhongrui Cai1 Fan Zheng4 Tianyou Ma1 Junhao Shen1,2 Haiteng Zhao1 Duanyang Zhang5 Huilun Zhang6 Kuikun Liu Chengqi Lyu Yanhui Duan1 Chiyu Chen1 Ningsheng Ma1 Jianfei Gao1 Han Lyu Dahua Lin1,3 Kai Chen1 |


We print the results to a html file.

In [30]:
page = f"<html> <head> <h1>Daily Dose of AI Research</h1> <h4>{date.today()}</h4> <p><i>Summaries generated with: {LLM}</i>"
with open("papers.html", "w") as f:
    f.write(page)
for paper in papers:
    page = f'<h2><a href="{paper["url"]}">{paper["title"]}</a></h2> <p>{paper["summary"]}</p>'
    with open("papers.html", "a") as f:
        f.write(page)
end = "</head>  </html>"
with open("papers.html", "a") as f:
    f.write(end)

We can also print the results to this notebook as markdown.

In [31]:
for paper in papers:
    printmd("**[{}]({})**<br>{}<br><br>".format(paper["title"],
                                                paper["url"],
                                                paper["summary"]))

**[T-pro 2.0: An Efficient Russian Hybrid-Reasoning Model and Playground](https://arxiv.org/pdf/2512.10430)**<br>T-pro 2.0: An Efficient Russian Hybrid-Reasoning Model and Playground Gen-T Team T-Tech, Moscow, Russia Correspondence: anatolii.s.potapov@gmail.com Abstract We introduceT-pro 2.0, an open-weight Rus- sian LLM for hybrid reasoning and efficient inference. The model supports direct answer- ing and reasoning-trace generation, using a Cyrillic-dense tokenizer and an adapted EAGLE-style speculative-decoding pipeline to reduce latency. To enable reproducible and extensi- ble research, we release the model weights, the T-Wix<br><br>